# Vectorization

Vectorization is a style of computer programming where operations are applied to multiple elements at the same time instead of individual elements.

Vector code has special registers and special instructions.

Most registers are between 8 and 64 bits

Vector registers are between 128 and 512 bits currently.

The vector instructions treat these bits as representing a vector of multiple elements of data and perform the operation element wise.

For example, take a 128 bit register.

We can think of these 128 bits as representing four 32-bit elements.

We can then compare multiple registers each containing these four elements and get the results of all four comparisons at once

![vector_operation](images/vector_op.JPG "vector operation")

The instructions I am describing here is [pcmpeqd](https://www.intel.com/content/www/us/en/docs/intrinsics-guide/index.html#text=_mm_cmpeq_epi32&ig_expand=878)

Formally, given `a` and `b` each of which is a 128 bit register, output dest, a new 128 bit register calculated in the following way

```
FOR j := 0 to 3
	i := j*32
	dst[i+31:i] := ( a[i+31:i] == b[i+31:i] ) ? 0xFFFFFFFF : 0
ENDFOR
```

If you want to operate on 256 bit registers you need to use a different instructions, such as [vpcmpeqd](https://www.intel.com/content/www/us/en/docs/intrinsics-guide/index.html#text=_mm256_cmpeq_epi32&ig_expand=878,879)


Now let's look at a problem.

Given an array of data, we want to count how many ints are equal to the target.

Here is some simple code to count.
```c
long count_ints(int* data, long n, int target) {
    long count = 0;
    for (long i = 0; i < n; i++) {
        if (data[i] == target) {
            count+=1;
        }
    }
    return count;
}
```


To help us look at code and how it compiles we are going [Compiler Explorer](https://godbolt.org/)

This website allows you to, among other things, compile code online and view the assemble.

Lets start by looking at the assembly for this specific problem.

We can see it [https://godbolt.org/z/sdn6nx9Ph](https://godbolt.org/z/sdn6nx9Ph)

We can see that the compiler is able to do some vectorization for us

```assembly
# -O3 -mno-sse -fno-unroll-loops 
count_ints(int*, long, int):                     # @count_ints(int*, long, int)
        test    rsi, rsi
        jle     .LBB0_1
        xor     ecx, ecx
        xor     eax, eax
.LBB0_4:                                # %for.body
        xor     r8d, r8d
        cmp     dword ptr [rdi + 4*rcx], edx
        sete    r8b
        add     rax, r8
        inc     rcx
        cmp     rsi, rcx
        jne     .LBB0_4
        ret
.LBB0_1:
        xor     eax, eax
        ret
```


```assembly
# -O3 -mno-sse
count_ints(int*, long, int):                     # @count_ints(int*, long, int)
        test    rsi, rsi
        jle     .LBB0_1
        mov     ecx, esi
        and     ecx, 3
        cmp     rsi, 4
        jae     .LBB0_8
        xor     r8d, r8d
        xor     eax, eax
        jmp     .LBB0_4
.LBB0_1:
        xor     eax, eax
        ret
.LBB0_8:                                # %for.body.preheader.new
        and     rsi, -4
        xor     r8d, r8d
        xor     eax, eax
.LBB0_9:                                # %for.body
        xor     r9d, r9d
        cmp     dword ptr [rdi + 4*r8], edx
        sete    r9b
        add     r9, rax
        xor     eax, eax
        cmp     dword ptr [rdi + 4*r8 + 4], edx
        sete    al
        xor     r10d, r10d
        cmp     dword ptr [rdi + 4*r8 + 8], edx
        sete    r10b
        add     r10, rax
        add     r10, r9
        xor     eax, eax
        cmp     dword ptr [rdi + 4*r8 + 12], edx
        sete    al
        add     rax, r10
        add     r8, 4
        cmp     rsi, r8
        jne     .LBB0_9
.LBB0_4:                                # %for.cond.cleanup.loopexit.unr-lcssa
        test    rcx, rcx
        je      .LBB0_7
        lea     rsi, [rdi + 4*r8]
        xor     edi, edi
.LBB0_6:                                # %for.body.epil
        xor     r8d, r8d
        cmp     dword ptr [rsi + 4*rdi], edx
        sete    r8b
        add     rax, r8
        inc     rdi
        cmp     rcx, rdi
        jne     .LBB0_6
.LBB0_7:                                # %for.cond.cleanup
        ret

```

```assembly
#-O3 -mavx2  -fno-unroll-loops 
.LCPI0_0:
        .quad   1                               # 0x1
count_ints(int*, long, int):                     # @count_ints(int*, long, int)
        test    rsi, rsi
        jle     .LBB0_1
        cmp     rsi, 4
        jae     .LBB0_5
        xor     ecx, ecx
        xor     eax, eax
        jmp     .LBB0_8
.LBB0_1:
        xor     eax, eax
        ret
.LBB0_5:                                # %vector.ph
        mov     rcx, rsi
        and     rcx, -4
        vmovd   xmm0, edx
        vpbroadcastd    xmm1, xmm0
        vpxor   xmm0, xmm0, xmm0
        xor     eax, eax
        vpbroadcastq    ymm2, qword ptr [rip + .LCPI0_0] # ymm2 = [1,1,1,1]
.LBB0_6:                                # %vector.body
        vpcmpeqd        xmm3, xmm1, xmmword ptr [rdi + 4*rax]
        vpmovzxdq       ymm3, xmm3              # ymm3 = xmm3[0],zero,xmm3[1],zero,xmm3[2],zero,xmm3[3],zero
        vpand   ymm3, ymm3, ymm2
        vpaddq  ymm0, ymm0, ymm3
        add     rax, 4
        cmp     rcx, rax
        jne     .LBB0_6
        vextracti128    xmm1, ymm0, 1
        vpaddq  xmm0, xmm0, xmm1
        vpshufd xmm1, xmm0, 238                 # xmm1 = xmm0[2,3,2,3]
        vpaddq  xmm0, xmm0, xmm1
        vmovq   rax, xmm0
        cmp     rcx, rsi
        je      .LBB0_2
.LBB0_8:                                # %for.body
        xor     r8d, r8d
        cmp     dword ptr [rdi + 4*rcx], edx
        sete    r8b
        add     rax, r8
        inc     rcx
        cmp     rsi, rcx
        jne     .LBB0_8
.LBB0_2:                                # %for.cond.cleanup
        vzeroupper
        ret

```

If you want to vectorization manually you can use what are known as [intrinsics](https://www.intel.com/content/www/us/en/docs/intrinsics-guide/index.html) 

These are special functions in c and c++ which compile down to a single assembly instruction 

For example to use [vpcmpeqd](https://www.intel.com/content/www/us/en/docs/intrinsics-guide/index.html#text=_mm256_cmpeq_epi32&ig_expand=878,879) and compare 8 32 bit integers at once we can use 

`__m256i _mm256_cmpeq_epi32 (__m256i a, __m256i b)`

Now let's think about how we might want to approach this function.

We are going to want to compare 8 elements at a time.  This means our main loop with increment by 8 each time.  This means we will also have to perform work at the end to deal with any extra iterations.  Here is a skeleton

```c
long count_ints(int* data, long n, int target) {
  long count = 0;
  // find the end of the main loop which is divisible by 8
  long clean_end = (n / 8) * 8;
  // move the i outside the loop so we can use it for the cleanup loop
  long i = 0;
  for (; i < clean_end; i += 8) {
    // compare the 8 elements at positions data[i] through data[i+7]
    int number_equal = ???;
    count += number_equal
  }
  // clean up and perform the last few iterations 
  for (; i < n; i++) {
    if (data[i] == target) {
      count += 1;
    }
  }
  return count;
}
```

Now it is just a matter of calculating the actual comparison.

The function we were looking at before will form the bases 

```
__m256i _mm256_cmpeq_epi32 (__m256i a, __m256i b)

FOR j := 0 to 7
	i := j*32
	dst[i+31:i] := ( a[i+31:i] == b[i+31:i] ) ? 0xFFFFFFFF : 0
ENDFOR
dst[MAX:256] := 0
```

So now we also need to get our comparisons into two vector registers.

The first will just loading the data from the array

The instruction we want is 
```
__m256i _mm256_loadu_si256 (__m256i const * mem_addr)
dst[255:0] := MEM[mem_addr+255:mem_addr]
dst[MAX:256] := 0
```

The next we want is is the target int, we already have it in one int, we just need it in all locations in a vector register

```
__m256i _mm256_set1_epi32 (int a)

FOR j := 0 to 7
	i := j*32
	dst[i+31:i] := a[31:0]
ENDFOR
dst[MAX:256] := 0
```

The last step is how to get the result back out of a vector register and into a normal int.

For this we are going to use 
```
int _mm256_movemask_epi8 (__m256i a)

FOR j := 0 to 31
	i := j*8
	dst[j] := a[i+7]
ENDFOR
```

Note that this is for each byte and not for each int, this is because there simply isn't an instruction for `_mm256_movemask_epi32`.

This gives us a bit mask, we will then need to count the bits in the mask

```
int _popcnt32 (int a)

dst := 0
FOR i := 0 to 31
	IF a[i]
		dst := dst + 1
	FI
ENDFOR
```

This will tell us how many bits are set in an integer.

So if we put it all together we can fill in the function as follows 


```c

long count_ints(int* data, long n, int target) {
  long count = 0;
  long clean_end = (n / 8) * 8;
  long i = 0;
  for (; i < clean_end; i += 8) {
      auto cmp_vec = _mm256_cmpeq_epi32(
        _mm256_loadu_si256((__m256i *)(data + i)),
                                    _mm256_set1_epi32(target));
        int movemask = _mm256_movemask_epi8(cmp_vec);
        count += _mm_popcnt_u32(movemask);
  }
  // we need this division to handle the fact we pulled out the bytes not the ints, but we can pull it out of the loop
  count /=4;
  for (; i < n; i++) {
    if (data[i] == target) {
      count += 1;
    }
  }
  return count;
}

```

This code and the assembly can be seen [https://godbolt.org/z/zdaac8fsT](https://godbolt.org/z/zdaac8fsT)


Let's take a look at the inner loop
```
        vpcmpeqd        ymm1, ymm0, ymmword ptr [rdi + 4*rcx]
        vpmovmskb       r9d, ymm1
        popcnt  r9d, r9d
        add     rax, r9
        add     rcx, 8
        cmp     rcx, r8
        jl      .LBB0_3
```

How long does this take to run?  Unlike most normal instructions, many vector operations take multiple cycles to run, and this cost is different on different cpus.

For example:

Vpcmpeqd 

| Architecture | latency | throughout |
|--- |--- |--- |
| all | 1 | .5 |

Vpmovmskb
| Architecture | latency | throughout |
|--- |--- |--- |
|Alderlake |4| 1|
|Icelake Xeon |3| 1|
|Sapphire Rapids |4| 1|
|Skylake |2| 1|

popcnt
| Architecture | latency | throughout |
|--- |--- |--- |
| all | 3 | 1 |

So this part takes 6 to 8 cycles depending on the architecture, not counting the loop overheads

Can we do better?  One thing to notice is that it is expensive to move data between vector registers and normal registers, so it is beneficial to limit how often you do this.

For example, the comparisons already give us -1 in each position if its equal and 0 if not, so we don't we just subtract these numbers to count up how many there are

```c
long count_ints(int* data, long n, int target) {
  long count = 0;
  long clean_end = (n / 8) * 8;
  long i = 0;

  auto compare = _mm256_set1_epi32(target);
  // the counts is now a vector
  auto counts = _mm256_setzero_si256();
  for (; i < clean_end; i += 8) {
      // we subtract the new comparison each time
      // this subtracts -1 (adds 1) if its equal, else subtracts 0
      counts = _mm256_sub_epi32(
          counts, _mm256_cmpeq_epi32(_mm256_loadu_si256((__m256i *)(data + i)),
                                    compare));
  }
  // afterword we need to pull out the different fields, but this is only at the end
    count += _mm256_extract_epi32(counts, 0);
    count += _mm256_extract_epi32(counts, 1);
    count += _mm256_extract_epi32(counts, 2);
    count += _mm256_extract_epi32(counts, 3);
    count += _mm256_extract_epi32(counts, 4);
    count += _mm256_extract_epi32(counts, 5);
    count += _mm256_extract_epi32(counts, 6);
    count += _mm256_extract_epi32(counts, 7);

  for (; i < n; i++) {
    if (data[i] == target) {
      count += 1;
    }
  }
  return count;
}
```

Now out inner loop looks like 
```
        vpcmpeqd        ymm3, ymm2, ymmword ptr [rdi + 4*rcx]
        vpsubd  ymm1, ymm1, ymm3
        add     rcx, 8
        cmp     rcx, rax
        jl      .LBB0_3
```

Vpcmpeqd 

| Architecture | latency | throughout |
|--- |--- |--- |
| all | 1 | .5 |

Vpsubd 

| Architecture | latency | throughout |
|--- |--- |--- |
| all | 1 | .33 |

This gives a total of just 2, plus the loop overhead.

Now there is a bug in this code.

What is it?

Try it yourself [http://preview.speedcode.org/ide/index.html?test_1_vector_count](http://preview.speedcode.org/ide/index.html?test_1_vector_count)

This is the same problem, except that you are going to count the bytes, instead of the ints.

The two different boxes have different optimizations applied to them in `CountNoOpt` the compiler will not perform any loop unrolling or vectorization in `CountOpt` it will.

Start with `CountNoOpt` and when you are able to make it faster than the reference move on the `CountOpt` and see if you can beat `opt serial`, then parallelize it to make it even faster
